In [1]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
print(pd.read_csv("/content/drive/MyDrive/IntrusionDetection/NSL-KDD/01-RawDataset/KDDTrain+.txt",header=None).shape)
print(pd.read_csv("/content/drive/MyDrive/IntrusionDetection/NSL-KDD/01-RawDataset/KDDTest+.txt",header=None).shape)

(125973, 43)
(22544, 43)


In [ ]:
def dados_tratar(df):

    #Renomear
    df.columns = (['duration'
    ,'protocol_type'
    ,'service'
    ,'flag'
    ,'src_bytes'
    ,'dst_bytes'
    ,'land'
    ,'wrong_fragment'
    ,'urgent'
    ,'hot'
    ,'num_failed_logins'
    ,'logged_in'
    ,'num_compromised'
    ,'root_shell'
    ,'su_attempted'
    ,'num_root'
    ,'num_file_creations'
    ,'num_shells'
    ,'num_access_files'
    ,'num_outbound_cmds'
    ,'is_host_login'
    ,'is_guest_login'
    ,'count'
    ,'srv_count'
    ,'serror_rate'
    ,'srv_serror_rate'
    ,'rerror_rate'
    ,'srv_rerror_rate'
    ,'same_srv_rate'
    ,'diff_srv_rate'
    ,'srv_diff_host_rate'
    ,'dst_host_count'
    ,'dst_host_srv_count'
    ,'dst_host_same_srv_rate'
    ,'dst_host_diff_srv_rate'
    ,'dst_host_same_src_port_rate'
    ,'dst_host_srv_diff_host_rate'
    ,'dst_host_serror_rate'
    ,'dst_host_srv_serror_rate'
    ,'dst_host_rerror_rate'
    ,'dst_host_srv_rerror_rate'
    ,'attack'
    ,'level'])


    # Classificar os ataques diretamente e criar uma nova coluna
    dos_attacks = ['apache2','back','land','neptune','mailbomb','pod','processtable','smurf','teardrop','udpstorm','worm']
    probe_attacks = ['ipsweep','mscan','nmap','portsweep','saint','satan']
    privilege_attacks = ['buffer_overflow','loadmdoule','perl','ps','rootkit','sqlattack','xterm']
    access_attacks = ['ftp_write','guess_passwd','http_tunnel','imap','multihop','named','phf','sendmail','snmpgetattack','snmpguess','spy','warezclient','warezmaster','xclock','xsnoop']

    df['attack_cat'] = 'Normal'  # Inicialmente, todos os ataques são classificados como "Normal"
    df.loc[df['attack'].isin(dos_attacks), 'attack_cat'] = 'DoS'
    df.loc[df['attack'].isin(probe_attacks), 'attack_cat'] = 'Probe'
    df.loc[df['attack'].isin(privilege_attacks), 'attack_cat'] = 'Privilege'
    df.loc[df['attack'].isin(access_attacks), 'attack_cat'] = 'Access'

    # Dropar colunas não úteis
    df = df.drop(columns=['level','attack'])

    # Identificando as top 5 categorias para 'service'
    top_service = df['service'].value_counts().head(5).index.tolist()
    # Substituindo categorias não presentes nas top 5 por 'outros' para 'service'
    df.loc[~df['service'].isin(top_service), 'service'] = 'outros'

    # Aplicando one-hot encoding
    df_encoded = pd.get_dummies(df, columns=['protocol_type', 'flag', 'service'], dtype=int)
    df_final = df_encoded.fillna(0)

    return df_final


if __name__ == "__main__":
    # Obtém a hora atual antes da execução do programa
    pd.set_option('display.max_columns', None)
    hora_inicio = datetime.datetime.now()
    print("Loading dataset...")
    #Load Data
    df1=pd.read_csv("../Daniel/raw_data/KDDTrain+.txt",header=None)
    df2=pd.read_csv("../Daniel/raw_data/KDDTest+.txt",header=None)

    df = pd.concat([df1,df2])
    print(f'Dados importados com shape: {df.shape}')
    df  = dados_tratar(df)
    print("Distribuição do label:")
    print(df['attack_cat'].value_counts())
    print(f'Dados para AL com shape: {df.shape}')
    print("Salvar dataset")
    df.drop("attack_cat", axis=1).to_csv('../Daniel/datasets/nslkdd_features.csv', index=False,header=False )
    df["attack_cat"].to_csv('../Daniel/datasets/nslkdd_labels.csv', index=False,header=False )
    print("Salvar dataset... DONE")

Loading dataset...
Dados importados com shape: (148517, 43)
Distribuição do label:
attack_cat
Normal       77207
DoS          53387
Probe        14077
Access        3738
Privilege      108
Name: count, dtype: int64
Dados para AL com shape: (148517, 59)
Salvar dataset
Salvar dataset... DONE
